# Day 24

## Part 1

In [689]:
Army = ( "Immune system", "Infection" )

# Not really needed, but ease visualisation for debugging purposes
Attack = {}
Attack['slashing'] = "S"
Attack['fire']     = "F"
Attack['radiation'] = "R"
Attack['bludgeoning'] = "B"
Attack['cold'] = "C"

class Group:
    def __init__(self,army,nunits,hp,immune,weak,damage,attack,initiative,ID=0):
        self.ID = ID
        self.army = army
        self.nunits = nunits
        self.hp = hp
        self.immune = immune
        self.weak = weak
        self.damage = damage 
        self.attack = attack
        self.initiative = initiative
        self.engaged = False
        self.target = None
        
    def __repr__(self):
        return "{:13s} {:2d}; N = {:4d} ; HP = {:5d} ; EP = {:5d} ; I = {:2d}; A = {} ; W = {} ; IM = {} ".format(
            Army[self.army],self.ID,self.nunits, self.hp,self.effpow(),self.initiative,self.attack,
            self.weak,self.immune
        )
            
    def effpow(self):
        return self.nunits*self.damage
    
    def damageMult(self,attack):
        if attack in self.immune:
            return 0
        elif attack in self.weak:
            return 2
        else:
            return 1

In [601]:
import re

def getArmies(filename):
    with open(filename) as f:
        blocks = [ [ l for l in b.split("\n") ] for b in f.read().split("\n\n") ] # split immune system and infection blocks, then split lines
    armies = [] # Immune system and infection
    ia = 0
    for b in blocks:
        army = []
        ii = 1
        for l in b[1:]: # ignore block header
            if len(l): # skip last empy line, if any
                bb = []
                if "(" in l:
                    bb = re.split(' \(|\) ', l) # split at ( or ) to isolate immune/weak block
                else:
                    bb = l.replace("with an attack ","|with an attack "). split("|") # some groups don't have immune/weak block
                nunits = int(bb[0].split(" ")[0])
                hp = int(bb[0].split(" ")[4])
                weak = []
                immune = []
                if len(bb)==3: # has immune/weak block
                    for iw in bb[1].split("; "):
                        if iw[:6] == "immune":
                            #immune = iw.replace("immune to ","").split(", ")
                            immune = [ Attack[i] for i in iw.replace("immune to ","").split(", ") ] # short attack name
                        else:
                            #weak = iw.replace("weak to ","").split(", ")
                            weak = [ Attack[i] for i in iw.replace("weak to ","").split(", ") ] # short attack name
                damage = int(bb[-1].split(" ")[5])
                #attack = bb[-1].split(" ")[6]
                attack = Attack[bb[-1].split(" ")[6]] # short attack name
                initiative = int(bb[-1].split(" ")[-1])
                group = Group(ia,nunits,hp,immune,weak,damage,attack,initiative,ID=ii)                 
                army.append(group)
                ii += 1
        armies.append(army)
        ia += 1
    return armies

In [602]:
armies = getArmies("data/day24test.txt")
armies

[[Immune system  1; N =   17 ; HP =  5390 ; EP = 76619 ; I =  2; A = F ; W = ['R', 'B'] ; IM = [] ,
  Immune system  2; N =  989 ; HP =  1274 ; EP = 24725 ; I =  3; A = S ; W = ['B', 'S'] ; IM = ['F'] ],
 [Infection      1; N =  801 ; HP =  4706 ; EP = 92916 ; I =  1; A = B ; W = ['R'] ; IM = [] ,
  Infection      2; N = 4485 ; HP =  2961 ; EP = 53820 ; I =  4; A = S ; W = ['F', 'C'] ; IM = ['R'] ]]

In [717]:
from collections import defaultdict

def fight(armies,boost=0,verbose=False,nturn=-1):

    iturn = 0
    
    if boost: # boost immune system
        for g in armies[0]:
            g.damage += boost
    
    killed = [-1,-1] 
    
    while True:
    
        if verbose:
            print("\n+++ Turn {} +++\n".format(iturn+1))
        
        ia = 0
        nu = []
        for a in armies:
            n = 0
            if verbose:
                print("{}:".format(Army[ia]))
            for g in a:
                if g.nunits>0:
                    if verbose:
                        print("Group {} contains {} units".format(g.ID,g.nunits))
                    n += g.nunits
            if n==0:
                if verbose:
                    print("No groups remain.")
            nu.append(n)
            ia += 1
        
        for ia in (0,1):
            if nu[ia]==0:
                return nu

        # end mexican standowns
        if sum(killed)==0: # nobdy has killed nobody
            if verbose:
                print("\n*** Mexican stand-down! ***")
            return nu
        
        # collecting the kills from both armies to solve mexican standowns!
        killed = [0,0] 
        
        if verbose:
            print("\n--- Selection phase ---")
            
        # selecting groups are sort by effective power, solving ties by initiatives
        groups = sorted(armies[0]+armies[1],key=lambda G: (G.effpow(),G.initiative),reverse=True)
        for G in groups:
            if G.nunits>0: # current group is still alive
                enemy = abs(G.army-1) # among the enemies
                # target must still be alive and not already targeted, attacker must be able to deliver damage
                targets = sorted(
                    [ E for E in armies[enemy] if E.nunits>0 and not E.engaged and E.damageMult(G.attack)*G.effpow()>0 ], 
                        key=lambda e: ( e.damageMult(G.attack)*G.effpow(), e.effpow(), e.initiative ), 
                        reverse=True ) # sorted by maximum damage, then effective power, then initiative
                if len(targets): # if target available
                    if verbose:
                        for t in targets:
                            print("{} group {} would deal defending group {} {} damage".format(
                                Army[G.army],G.ID,t.ID,t.damageMult(G.attack)*G.effpow() ) )
                    G.target = targets[0]
                    targets[0].engaged = True
                else:
                    G.target = None

        if verbose:
            print("\n--- Attacking phase ---")

        # attacking according to initiative
        groups = sorted(armies[0]+armies[1],key=lambda G: G.initiative,reverse=True)
        for G in groups:
            if G.nunits>0 and G.target!=None:
                T = G.target
                damage = T.damageMult(G.attack)*G.effpow()
                nkill = min( damage // T.hp , T.nunits )
                killed[G.army] += nkill
                T.nunits -= nkill
                if verbose:
                    print("{} group {} attacks defending group {}, killing {} units".format(
                                Army[G.army],G.ID,T.ID,nkill))   
        
        # release *all* groups from engagement for next round
        # it has to happen *here* and not in combat loop, since some groups might have a target 
        # but might not attack becouse they died in previous combat!
        for G in armies[0]+armies[1]:
            G.engaged = False  
        
        iturn += 1
        if iturn==nturn:
            break

In [718]:
armies = getArmies("data/day24test.txt")

survivors = fight(armies,verbose=True)
print("Survivors =",sum(survivors))


+++ Turn 1 +++

Immune system:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

--- Selection phase ---
Infection group 1 would deal defending group 1 185832 damage
Infection group 1 would deal defending group 2 185832 damage
Immune system group 1 would deal defending group 2 153238 damage
Immune system group 1 would deal defending group 1 76619 damage
Infection group 2 would deal defending group 2 107640 damage
Immune system group 2 would deal defending group 1 24725 damage

--- Attacking phase ---
Infection group 2 attacks defending group 2, killing 84 units
Immune system group 2 attacks defending group 1, killing 4 units
Immune system group 1 attacks defending group 2, killing 51 units
Infection group 1 attacks defending group 1, killing 17 units

+++ Turn 2 +++

Immune system:
Group 2 contains 905 units
Infection:
Group 1 contains 797 units
Group 2 contains 4434 units

--- Selection phase ---
Infection group 1 

In [719]:
armies = getArmies("data/input24.txt")
armies

[[Immune system  1; N = 5711 ; HP =  6662 ; EP = 51399 ; I = 14; A = B ; W = ['S'] ; IM = ['F'] ,
  Immune system  2; N = 2108 ; HP =  8185 ; EP = 75888 ; I = 13; A = S ; W = ['R', 'B'] ; IM = [] ,
  Immune system  3; N = 1590 ; HP =  3940 ; EP = 38160 ; I =  5; A = C ; W = [] ; IM = [] ,
  Immune system  4; N = 2546 ; HP =  6960 ; EP = 63650 ; I =  2; A = S ; W = [] ; IM = [] ,
  Immune system  5; N = 1084 ; HP =  3450 ; EP = 29268 ; I = 11; A = S ; W = [] ; IM = ['B'] ,
  Immune system  6; N =  265 ; HP =  8223 ; EP = 68635 ; I = 12; A = C ; W = [] ; IM = ['R', 'B', 'C'] ,
  Immune system  7; N = 6792 ; HP =  6242 ; EP = 61128 ; I = 18; A = S ; W = ['B', 'R'] ; IM = ['S'] ,
  Immune system  8; N = 3336 ; HP = 12681 ; EP = 93408 ; I =  6; A = F ; W = ['S'] ; IM = [] ,
  Immune system  9; N =  752 ; HP =  5272 ; EP = 51888 ; I =  4; A = R ; W = ['B', 'R'] ; IM = ['S'] ,
  Immune system 10; N =   96 ; HP =  7266 ; EP = 70848 ; I =  8; A = B ; W = [] ; IM = ['F'] ],
 [Infection      1; N

In [720]:
armies = getArmies("data/input24.txt")
survivors = fight(armies)
print("Survivors =",sum(survivors))

Survivors = 38008


## Part 2

Simple brute force search for minimal boost value seems to work fine with example, but full input get stuck at boost = 28.

Looking at the verbose output, it seems that there are cases there the battle get stuck:

`--- Attacking phase ---`  
`Immune system group 6 attacks defending group 3, killing 0 units`  
`Infection group 5 attacks defending group 6, killing 0 units`  

I need to implement a stopping mechanism for those stalling matches.

In [726]:
armies = getArmies("data/day24test.txt")
survivors = fight(armies,boost=1570,verbose=False)
print("Survivors =",survivors)

Survivors = [51, 0]


In [727]:
from copy import deepcopy

def getBoost(armies):
    boost = 0
    while True:
        survivors = fight(deepcopy(armies),boost=boost)
        if survivors[0]>0 and survivors[1]==0: # avoid mexican standdowns
            print("Boost = {} -> Immune system survivors = {}".format(boost,survivors[0]))
            return(boost)
        boost += 1

In [728]:
armies = getArmies("data/day24test.txt")
getBoost(armies)

Boost = 1570 -> Immune system survivors = 51


1570

In [729]:
armies = getArmies("data/input24.txt")
fight(armies,boost=28,verbose=False)

[58, 18219]

In [730]:
armies = getArmies("data/input24.txt")
getBoost(armies)

Boost = 34 -> Immune system survivors = 4009


34